In [0]:
df=spark.read.table("jpmc.nara_bronze.sales")

In [0]:
df_new=df.dropDuplicates().dropna().drop("ingestion_date")

In [0]:
df_new.write.mode("overwrite").saveAsTable("jpmc.nara_silver.sales_stg")

In [0]:
display(df_new)

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
create or replace view jpmc.nara_gold.top_3_cust as 
select customer_id, round(sum(total_amount),2) as total_amount
from jpmc.nara_silver.sales_stg
group by customer_id
order by total_amount desc
limit 3

In [0]:
%sql
select * from  jpmc.nara_gold.top_3_cust

Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.sql.functions import *
df_insights = (
    spark.table("jpmc.nara_silver.sales_stg")
    .selectExpr(
        "customer_id",
        "total_amount"
    )
    .agg(
        expr("count(distinct customer_id)").alias("total_customers"),
        expr("round(sum(total_amount), 2)").alias("total_sales"),
        expr("round(avg(total_amount), 2)").alias("average_sales_per_transaction"),
        expr("max(total_amount)").alias("highest_single_transaction"),
        expr("min(total_amount)").alias("lowest_single_transaction")
    )
)
df_insights.createOrReplaceTempView("jpmc.nara_gold.insights")